# Intro, Imaging & Machine Learning in PyCrystEM

PyCrystEM is an open-source library for crystallographic electron microscopy. Multi-dimensional data processing tools build on the HyperSpy library.

This notebook demonstrates the use of PyCrystEM to: form so called 'virtual diffraction images' by plotting the diffracted intensity in a particular pixel in the reciprocal space images, learn the component patterns making up the data using machine learning techniques, and perform data enhancement as a pre-processing step for further analysis such as indexation and orientation mapping. 

## Change Log

22/03/18 Duncan Johnstone & Phillip Crout - Updated for ePSIC Hyperspy Workshop

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

# Requirements

pyXem 0.5

HyperSpy 1.3

PyMatGen

## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding & Vector Analysis</a>
6. <a href='#ori'> Orientation Mapping</a>

Import pyXem

In [1]:
%matplotlib tk
import pyxem as pxm

## <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [2]:
dp = pxm.load('/home/hremadmin/nanowire_precession.hdf5')
dp = pxm.ElectronDiffraction(dp)

Look at what kind of object 'dp' is

In [3]:
dp

<ElectronDiffraction, title: , dimensions: (30, 100|144, 144)>

Inspect the metadata associated with the object 'dp'

In [4]:
dp.metadata

├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

Set important experimental parameters using the built in function

In [5]:
dp.set_experimental_parameters(accelerating_voltage=300.,
                               camera_length=21.,
                               scan_rotation=277.,
                               convergence_angle=0.7,
                               exposure_time=10.)

See how this changed the metadata

In [6]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── accelerating_voltage = 300.0
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

Set another metadata item and check it

In [7]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── accelerating_voltage = 300.0
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── General
│   └── title = GaAs Nanowire
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

Plot the data to inspect it

In [8]:
dp.plot()

Set the diffraction pattern calibration

In [9]:
dp.set_diffraction_calibration(0.0035)
dp.set_scan_calibration(10)

Plot the data with the new calibration

In [10]:
dp.plot()

## <a id='cal'></a> 2. Alignment & Calibration

Many diffraction pattern analyses depend on knowing the location of the center of the diffraction pattern. It's easiest if this is in the center of the data array and if any shifts throughout a scan are corrected before any further work is done. Alignment and centering should therefore be done before anything else.

## <a id='vdf'></a> 3. Virtual Diffraction Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [11]:
roi = pxm.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.02)
dp.plot_interactive_virtual_image(roi=roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [12]:
vdf=dp.get_virtual_image(roi)

Plot the virtual dark-field image

In [14]:
vdf.plot()

Save the virtual dark-field image as a 32bit tif

In [15]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

Plot the data with an adjustable marker indicating where to crop the scan region

In [18]:
reg = pxm.roi.RectangularROI(left=50, top=50, right=100, bottom=100)
dp.plot()
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [19]:
dpc = reg(dp)

In [20]:
dpc

<ElectronDiffraction, title: GaAs Nanowire, dimensions: (24, 24|144, 144)>

# <a id='ml'></a> 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

In [21]:
dpc.change_dtype('float')
dpc.decomposition(True, algorithm='svd')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [22]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [23]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [24]:
dpc.decomposition(True, algorithm='nmf', output_dimension=3)

Plot the NMF results

In [25]:
dpc.plot_decomposition_results()

# Pattern enhancement & Background Subtraction

Perform a background subtraction by radially integrating the 2D diffraction pattern and fitting a model containing a lorentzian, exponential and linear function

In [ ]:
dp.remove_background(1).plot()

Obtain a radially integrated diffraction profile and plot it

In [ ]:
rp = dp.get_radial_profile()
rp.plot()

# <a id='vec'></a> 5. Peak Finding & Vector Analysis

Interactively tune peak finding parameters

In [33]:
dp.find_peaks_interactive()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Perform peak finding on all diffraction patterns in data

In [34]:
peaks = dp.find_peaks()

A Jupyter Widget

A Jupyter Widget

Check the peaks object

In [35]:
peaks

<DiffractionVectors, title: , dimensions: (30, 100|)>

Look at what's in the peaks object

In [36]:
peaks.data

array([[ array([[ 0.245 ,  0.2415],
       [ 0.245 ,  0.238 ],
       [ 0.245 ,  0.1995],
       ..., 
       [-0.2485, -0.112 ],
       [-0.2485, -0.1225],
       [-0.2485, -0.168 ]]),
        array([[ 0.245 ,  0.238 ],
       [ 0.245 ,  0.168 ],
       [ 0.245 ,  0.133 ],
       ..., 
       [-0.2485, -0.196 ],
       [-0.2485, -0.21  ],
       [-0.2485, -0.2345]]),
        array([[ 0.245 ,  0.196 ],
       [ 0.245 ,  0.168 ],
       [ 0.245 ,  0.126 ],
       ..., 
       [-0.2485, -0.2135],
       [-0.2485, -0.2275],
       [-0.2485, -0.2485]]),
        ...,
        array([[ 0.245 ,  0.2135],
       [ 0.245 ,  0.1785],
       [ 0.245 ,  0.1645],
       ..., 
       [-0.2485, -0.203 ],
       [-0.2485, -0.2135],
       [-0.2485, -0.2345]]),
        array([[ 0.245 ,  0.2345],
       [ 0.245 ,  0.2135],
       [ 0.245 ,  0.203 ],
       ..., 
       [-0.2485, -0.203 ],
       [-0.2485, -0.217 ],
       [-0.2485, -0.2485]]),
        array([[ 0.245 ,  0.217 ],
       [ 0.245 ,  0.182 ],

# <a id='ori'></a> 6. Orientation Mapping

In [41]:
import numpy as np
import pymatgen as pmg
from pyxem.utils.sim_utils import equispaced_so3_grid

Build crystal structure

In [42]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.cubic(5.6535)

In [43]:
structure = pmg.Structure.from_spacegroup("F23",lattice, [Ga,As], [[0, 0, 0],[0.25,0.25,0.25]])

Set up diffraction simulator and library generator

In [44]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)

In [45]:
recip_d_111 = np.sqrt((3/5.6535**2))

In [46]:
rot_list = equispaced_so3_grid(90,180,90,5)
struc_lib = dict()
struc_lib["A"] = (structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=recip_d_111/11.4,
                                            reciprocal_radius=0.75,
                                            half_shape=(72,72),
                                            representation='euler',
                                            with_direct_beam=False)

In [ ]:
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate()